## Unsupervised outlier detection using LSTM + AE from TS data 

- Data Measuring Multiple gases with an environmental sensors  
- Place : the computer center of Soongsil Unv.  
- Time : 2022-05-01 ~ 2022-05-19  
- Feature : TMP ,HMD, TVOC, CO, CO2, CH2O, PM10 (7 features)  

## 0. Setting

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# %cd '/content/drive/MyDrive/Outlier Detection Paper'
# !ls

### Library Call

In [ ]:
# 상용 라이브러리
from glob import glob
import os
import pandas as pd
import numpy as np
from datetime import datetime
import time
import pickle

# 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib
import plotly.express as px
import plotly.graph_objects as go

# 한글 폰트 패치
matplotlib.rcParams['font.family']='Malgun Gothic'
matplotlib.rcParams['axes.unicode_minus'] = False   

# 시각화 포맷 설정
plt.style.use("ggplot")
sns.set(font_scale=1.5)
sns.set_style("whitegrid")
sns.set_context("talk")

# 경고문 처리
import warnings
warnings.filterwarnings('ignore')

# Sckit-Learn 라이브러리
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import OneClassSVM
from sklearn.cluster import KMeans, AgglomerativeClustering, DBSCAN
from sklearn.mixture import GaussianMixture

# Statistic Tools 라이브러리
from statsmodels.tsa.stattools import adfuller, kpss
from scipy.cluster.hierarchy import dendrogram, linkage

# Tensorflow 라이브러리
import tensorflow as tf
from tensorflow import keras
from tqdm import tqdm
# import tensorflow_addons as tfa
from keras import backend
from functools import partial
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Activation, Flatten, Dense
from keras import layers, models
# from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping
# from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

### User Function Definition

In [ ]:
# BoxplotEDA Function Definition
def boxplotEDA(data,cols,nrow,ncol,title):
  df = data[cols]
  fig, ax = plt.subplots(nrow, ncol, figsize=(6*ncol,5*nrow))
  plt.suptitle(title,y=0.92,size=30)
  i=0
  for col in cols:
    sns.boxplot(y = df[col], ax=ax[i//ncol,i%ncol], palette='Set3', linewidth=1.5)
    i+=1  

# Sublineplot
def subplots(data,cols,nrow,ncol,title):
  df = data[cols]
  fig, ax = plt.subplots(nrow, ncol, figsize=(6*ncol,5*nrow))
  plt.suptitle(title, y=0.92, size=30)
  i=0
  for col in cols:
    ax[i//ncol,i%ncol].plot(df[col])
    ax[i//ncol,i%ncol].set_ylabel(col)
    i+=1  
  
# lineplot
def lineplot(data,cols,title):
  df = data[cols]
  plt.figure(figsize=(20,10))
  plt.title(title, y=1.05, size=25)
  for col in cols:
    plt.plot(df[col], label=col)
  plt.legend(loc='upper right')  

# corr_matrix
def corr_matrix(data, cols, title):
  df = data[cols]
  colormap = plt.cm.PuBu 
  plt.figure(figsize=(12, 12)) 
  plt.title(title, y=1.05, size=20)
  sns.heatmap(df.astype(float).corr(), linewidths = 0.1, vmax = 1.0,
              square = True, cmap = colormap, linecolor = "white", annot = True, fmt='.2f',
              annot_kws = {"size" : 12})
  plt.show()

# IQR Based Outlier Processing Function Definition
def outliers_iqr(data):
  q1,q3 = np.percentile(data,[25,75])
  iqr=q3-q1
  lower_bound=  q1 - (iqr *1.5)
  upper_bound = q3 + (iqr *1.5)
  data[data>upper_bound] = np.nan #np.mean(data)
  data[data<lower_bound] = np.nan #np.mean(data)
  data.interpolate(method="ffill", inplace=True)
  data.interpolate(method="bfill", inplace=True)
  return data

# Reconstruction Error Computation Function
def RE_SCORE(X_input, X_pred):
    score = pd.DataFrame(index = X_pred.index)
    score['RE_SCORE'] = np.mean(np.square(X_input-X_pred),axis=1)
    return score

# Simulation Data Cleansing Function Definition
def simul_cleansing(df):
  df = df[['TsYMD','Temperature','Humidity','TVOC','CO','CO2','CH2O','PM10']]
  df['TsYMD'] = pd.to_datetime(df['TsYMD'])
  df = df.sort_values('TsYMD')
  df['TsYMD'] = df['TsYMD'].dt.to_period(freq='min')
  df.rename(columns={'TsYMD':'Time'},inplace=True)
  time =df['Time'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
  df.index = df['Time']
  df.drop('Time',axis=1,inplace=True)
  return df

## 1. Data Load

### Device 11

In [ ]:
# glob Function File Loading
filst = sorted(glob('data/*.csv'))
filst

In [ ]:
# Sensor Data Load
device11 = pd.read_csv(filst[0])
device16 = pd.read_csv(filst[1])

In [ ]:
print(device11.shape)
device11.head()

In [ ]:
# Device11 Information
device11.info()

In [ ]:
# Device11 Description
device11.describe()

In [ ]:
# Device11 Boxplot
boxplotEDA(device11,device11.columns[3:],4,4,'Device 11 Boxplot')

In [ ]:
# Device11 Subplots
subplots(device11,device11.columns[3:],4,4,'Device 11 Subplots')

In [ ]:
# Device11 Lineplot
lineplot(device11, device11.columns[3:],'Device 11 Lineplot')

In [ ]:
# Device11 Correlation Heatmap
corr_matrix(device11, device11.columns[3:], 'Device 11 Correlation Heatmap')

### Device 16

In [ ]:
print(device16.shape)
device16.head()

In [ ]:
# Device16 Information
device16.info()

In [ ]:
# Device16 Description
device16.describe()

In [ ]:
# Device16 Boxplot
boxplotEDA(device16,device16.columns[3:],4,4,'Device 16 Boxplot')

In [ ]:
# Device16 Subplots
subplots(device16,device16.columns[3:],4,4,'Device 16 Suplots')

In [ ]:
# Device16 Lineplot
lineplot(device16, device16.columns[3:],'Device 16 Lineplot')

In [ ]:
# Device16 Correlation Heatmap except NaN
corr_matrix(device16, device16.columns[3:].difference(['Radioactivity','H2S']),'Device 16 Correlation Heatmap except NaN')

## 2. Data Prprocessing

### Data Feature Selection

In [ ]:
d11=device11.copy()
d16=device16.copy()

In [ ]:
d11.columns

In [ ]:
# Feature Selection : ['TsYMD','Temperature','Humidity','TVOC','CO','CO2','CH2O','PM10']
d11 = d11[['TsYMD','Temperature','Humidity','TVOC','CO','CO2','CH2O','PM10']]
d16 = d16[['TsYMD','Temperature','Humidity','TVOC','CO','CO2','CH2O','PM10']]
print('d11.shape :',d11.shape)
print('d16.shape :',d16.shape)

In [ ]:
# Device16 Correlation Heatmap
corr_matrix(d16, d16.columns[1:],'Device 16 Correlation Heatmap')

### Separate device because of long term over 2min


2022-04-13 04:27 | 2022-04-13 10:18 사이 공백 발생  
-> 공백 발생하는 곳마다 여러개의 dataframe으로 나눠 진행 필요

In [ ]:
d11['TsYMD'][4127:4131]
#d16['TsYMD'][8509:8513]

proceed from here

In [ ]:
d11['TsYMD']=pd.to_datetime(d11['TsYMD'])
d11=d11.sort_values('TsYMD')
d11['TsYMD']=d11['TsYMD'].dt.to_period(freq='min')
time =d11['TsYMD'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
time.head()

In [ ]:
check_list=[0]
for i in range(1,len(time)):
  now=datetime.strptime(time[i],'%Y%m%d%H%M')
  past=datetime.strptime(time[i-1],'%Y%m%d%H%M')
  diff=now-past
  diff=diff.seconds/60
  if diff >6:
    check_list.append(i)
check_list.append(len(time))

check_list

In [ ]:
for i in range(len(check_list)):
  print(d11['TsYMD'][check_list[i]-1:check_list[i]+1])

In [ ]:
d11_1=d11[check_list[0]:check_list[1]]
d11_2=d11[check_list[1]+1:check_list[2]]
d11_3=d11[check_list[2]+1:check_list[3]]
d11_4=d11[check_list[3]+1:check_list[4]]
d11_5=d11[check_list[4]+1:check_list[5]]

device11 -> d11_1, d11_2, d11_3, d11_4, d11_5로 나눠 진행

### Time Synchronization (2 min)

In [ ]:
import datetime

def deviceEDA(device):
  device.reset_index(drop=True,inplace=True)
  time =device['TsYMD'].apply(lambda x: x.strftime('%Y%m%d%H%M'))
  time.reset_index(drop=True,inplace=True)
  for i in range(len(device['TsYMD'])):
    if int(time[i][-2:])%2 ==1:
      device['TsYMD'][i]=device['TsYMD'][i]+datetime.timedelta(minutes=1)
  return device

def frameEDA(device):
  frame = pd.date_range(start = device['TsYMD'].iloc[0].strftime('%Y-%m-%d %H:%M'),            # 날짜 범위 시작
                     end = device['TsYMD'].iloc[-1].strftime('%Y-%m-%d %H:%M'),                # 날짜 범위 끝
                     freq = '2min',                           # 시간 간격( 2분 간격)
                     tz = 'Asia/Seoul')                       # 시간대(timezone)
  frame=pd.DataFrame(frame)
  frame.columns=['TsYMD']
  frame['TsYMD'] = frame['TsYMD'].dt.to_period(freq = 'min')  #분까지 끊기
  return frame

def mergeEDA(device,frame):
  merge_device = pd.merge(frame, device, how='outer',on='TsYMD')
  merge_device = merge_device.sort_values('TsYMD')
  merge_device.interpolate(method="ffill", inplace=True)
  return merge_device

def synchronization(device):
  new_device=deviceEDA(device)
  new_device_frame=frameEDA(new_device)
  new_device= mergeEDA(new_device,new_device_frame)
  return new_device

In [ ]:
newd11_1=synchronization(d11_1)
newd11_2=synchronization(d11_2)
newd11_3=synchronization(d11_3)
newd11_4=synchronization(d11_4)
newd11_5=synchronization(d11_5)

In [ ]:
print('-----------------------')
print(len(d11_1))
print(len(newd11_1))
print('-----------------------')
print(len(d11_2))
print(len(newd11_2))
print('-----------------------')
print(len(d11_3))
print(len(newd11_3))
print('-----------------------')
print(len(d11_4))
print(len(newd11_4))
print('-----------------------')
print(len(d11_5))
print(len(newd11_5))
print('-----------------------')

In [ ]:
d11_1=newd11_1
d11_2=newd11_2
d11_3=newd11_3
d11_4=newd11_4
d11_5=newd11_5

최종 device11 dataframe : d11_1, d11_2, d11_3, d11_4, d11_5

### Data Merge

In [ ]:
df_list = [d11_1,d11_2,d11_3,d11_4,d11_5]
df11 = pd.concat(df_list)
df11.rename(columns={'TsYMD':'Time'},inplace=True)
df11.drop_duplicates(subset=['Time'],inplace=True)
df11.reset_index(inplace=True,drop=True)
df11

### Difference

In [ ]:
def adf_test(timeseries, pvalue = .05, regression_option = 'ct'):
    print ('Results of Dickey-Fuller Test:')
    dftest = adfuller(timeseries, autolag='AIC', regression = regression_option)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    print (dfoutput)
    if dfoutput[1] < pvalue:
       print(f"정상시계열이 아니라는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 있으므로 해당 데이터는 정상성이 보장됩니다.")
    else:
       print(f"정상시계열이 아니라는 귀무가설을 {pvalue*100}%의 유의수준으로 기각할 수 없으므로 해당 데이터는 정상성을 보장하지 못합니다.")

def adf_test1(timeseries, pvalue = .05, regression_option = 'ct'):
    dftest = adfuller(timeseries, autolag='AIC', regression = regression_option)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    if dfoutput[1] < pvalue:
       print(timeseries.name,f"정상성이 보장됩니다.")
    else:
       print(timeseries.name,f"정상성을 보장하지 못합니다.")

In [ ]:
adf_test(df11['PM10'])

In [ ]:
for i in range(7):
  adf_test1(df11.iloc[:,1+i])

In the case of 'device11', Normality is satisfied in all areas except CO, NO2, H2S, and NH3.

In [ ]:
diff11=df11.copy()

In [ ]:
# all 1st Difference
for i in range(7):
    diff11.iloc[:,1+i]=diff11.iloc[:,1+i].replace(diff11.iloc[:,1+i].diff().dropna())
for i in range(7):
  adf_test1(diff11.iloc[:,1+i])

In [ ]:
diff11

In [ ]:
def diff1(timeseries, pvalue = .05, regression_option = 'ct'):
    dftest = adfuller(timeseries, autolag='AIC', regression = regression_option)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    if dfoutput[1] < pvalue:
       timeseries=timeseries
    else:
       timeseries=timeseries.replace(timeseries.diff(1).dropna())
       
def diff2(timeseries, pvalue = .05, regression_option = 'ct'):
    dftest = adfuller(timeseries, autolag='AIC', regression = regression_option)
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
       dfoutput['Critical Value (%s)'%key] = value
    if dfoutput[1] < pvalue:
       timeseries=timeseries
    else:
       timeseries=timeseries.replace(timeseries.diff().diff().dropna())

In [ ]:
for i in range(7):
    diff1(diff11.iloc[:,1+i])
    diff2(diff11.iloc[:,1+i])
for i in range(7):
    adf_test1(diff11.iloc[:,1+i])

### Outlier Process

In [ ]:
# Outlier Processing for 7 Features
for i in range(7):
    outliers_iqr(df11.iloc[:,1+i])

In [ ]:
# Outlier Processed Device 11 Boxplot
boxplotEDA(df11,df11.columns[1:],2,4,'Outlier Processed Device 11 Boxplot')

## 3. Modeling

### Modeling Preparation

In [ ]:
# DataFrame Time Indexing for Time Series Data
df11.set_index('Time',inplace=True)
diff11.set_index('Time',inplace=True)
print('df11 :',df11.shape)
print('diff11 :',diff11.shape)
df11.head()

In [ ]:
# train_test_split | 0.8 : 0.2
sep = int(df11.shape[0] * 0.8)
X_train = df11.iloc[:sep,:]
X_test = df11.iloc[sep:,:]
print('X_train.shape:',X_train.shape)
print('X_test.shape:',X_test.shape)

# Normalization
norm = MinMaxScaler()
X_train_scaled = norm.fit_transform(X_train)
X_test_scaled = norm.transform(X_test)

# Data Windowing
n_features = X_train_scaled.shape[1]   # should be 7
ws = 3                                 # window size

# keep only a multiple of ws rows
n_rows_trim = (X_train_scaled.shape[0] // ws) * ws
X_train_trim = X_train_scaled[:n_rows_trim]
X_test_trim  = X_test_scaled[: (X_test_scaled.shape[0] // ws) * ws]

train = X_train_trim.reshape(-1, ws, n_features)
test  = X_test_trim.reshape(-1, ws, n_features)


### LSTM-AE 

<b>Encoder

In [ ]:
# Encoder

encoder_input = keras.layers.Input(shape=(train.shape[1],train.shape[2]))

L1 = LSTM(128, activation='relu', return_sequences=True)(encoder_input)
L2 = LSTM(64, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(L1)
L3 = LSTM(32, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(L2)
L4 = LSTM(4, activation='relu', return_sequences=False, dropout=0.2, recurrent_dropout=0.2)(L3)
L5 = keras.layers.RepeatVector(train.shape[1])(L4)

encoder_output = L5

In [ ]:
encoder = keras.Model(encoder_input, encoder_output)
encoder.summary()

<b>Decoder

In [ ]:
# Decoder
decoder_input = keras.layers.Input(shape=(train.shape[1],4))

L6 = LSTM(4, activation='relu', return_sequences=True)(decoder_input)
L7 = LSTM(32, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(L6)
L8 = LSTM(64, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(L7)
L9 = LSTM(128, activation='relu', return_sequences=True, dropout=0.2, recurrent_dropout=0.2)(L8)
output = tf.keras.layers.TimeDistributed(Dense(train.shape[2]))(L9)

decoder_output = output

In [ ]:
decoder = keras.Model(decoder_input, decoder_output)
decoder.summary()

<b>Connecting Encoder & Decoder

In [ ]:
# LSTM-AE = Encoder + Decoder

encoder_in = keras.layers.Input(shape=(train.shape[1],train.shape[2]))
x = encoder(encoder_in)
decoder_out = decoder(x)

lstm_ae_model = keras.Model(encoder_in,decoder_out)
lstm_ae_model.compile(optimizer='adam', loss='mse')
lstm_ae_model.summary()

<b> LSTM-AE Model Training

In [ ]:
# Checkpoint Callback Function Definition
checkpoint_dir = './lstm-ae-checkpoint/'
checkpoint_path = checkpoint_dir + 'cp-{epoch:04d}-{val_loss:.2f}.weights.h5'

# Check val_loss change by 5 epochs - Stop learning if there is no change
early_stopping = EarlyStopping(monitor='val_loss', patience=10)
cp = ModelCheckpoint(filepath=checkpoint_path, verbose=1,
                     save_weights_only=True,
                     save_best_only=True)

In [ ]:
# TQDM Tracking LSTN-AE Model Training
nb_epochs = 30
batch_size = 10
# tqdm_callback = tfa.callbacks.TQDMProgressBar()
history = lstm_ae_model.fit(train, train, epochs=nb_epochs, batch_size=batch_size,
                    callbacks=[early_stopping, cp], validation_split=0.05).history

<b> LSTM-AE Model Save & Load

In [ ]:
# LSTM-AE 모델 저장
lstm_ae_model.save('./lstm_model_save/lstm_ae_0712_t3.h5')

# encoder 모델 저장
encoder.save('./lstm_model_save/encoder_0712_t3.h5')

# decoder 모델 저장
decoder.save('./lstm_model_save/decoder_0712_t3.h5')

In [ ]:
# Model Load
# lstm_ae_model = keras.models.load_model('./lstm_model_save/lstm_ae_0712_t3.h5')
# encoder = keras.models.load_model('./lstm_model_save/encoder_0712_t3.h5')
# decoder = keras.models.load_model('./lstm_model_save/decoder_0712_t3.h5')

<b>Model Performance Evaluation

In [ ]:
# Loss curve of the training_set
plt.figure(figsize=[6,4])
plt.plot(history['loss'], 'black', linewidth=2.0)
plt.plot(history['val_loss'], 'green', linewidth=2.0)
plt.legend(['Training Loss', 'Validation Loss'], fontsize=14)
plt.xlabel('Epochs', fontsize=10)
plt.ylabel('Loss', fontsize=10)
plt.title('Loss Curves', fontsize=12)

In [ ]:
# Original Train Scaled DataSet
og_train = pd.DataFrame(X_train_scaled, columns = X_train.columns)
og_train.index = X_train.index
og_train.head()

In [ ]:
# Prediction of Train DataSet
X_pred_train = lstm_ae_model.predict(train)

train_pred = X_pred_train.reshape(-1, X_pred_train.shape[2])
train_pred = pd.DataFrame(train_pred, columns = X_train.columns)
train_pred.index = X_train.index[: train_pred.shape[0] ]
# train_pred.index = X_train[:10587].index
print('train_pred.shape :',train_pred.shape)
train_pred.head()

In [ ]:
# Train data Reconstruction Error
train_score = RE_SCORE(X_train_scaled[:train_pred.shape[0]], train_pred)
train_score = train_score.set_index(train_score.index)
train_score.head()

In [ ]:
# Train Data Reconstruction Error Threshold
upper, lower= np.quantile(train_score['RE_SCORE'].values,0.75), np.quantile(train_score['RE_SCORE'].values,0.25)
iqr = upper-lower
train_boundary = upper + iqr*1.5
print('Train Data IQR Based boundary :',np.round(train_boundary,4))

In [ ]:
train_score['Threshold'] = train_boundary
train_score.head()

In [ ]:
# Reconstruction Error of Train Set
# fig = px.scatter(train_score, x=train_score.index, y='RE_SCORE', title='Reconstruction Error of Train Set')
# fig.add_trace(go.Scatter(x=train_score.index, y=train_score['Threshold'], name='Threshold',
#                          line=dict(width=5,dash='dash',color=('rgb(237,37,75)'))))
# Can't run this piece of cryptic code

In [ ]:
# Reconstruction Error Distribution of Train Set
fig = px.histogram(train_score, x="RE_SCORE", title ='Reconstruction Error Distribution of Train Set', marginal='box')
fig.add_vline(x=train_boundary, line_width=3, line_dash="dash", line_color="red",annotation_text="Threshold "+str(np.round(train_boundary,4)), annotation_position="bottom right")
fig.show()

In [ ]:
# Train Set Truth and Pred Error Comparation
# plt.figure(figsize=(16,6))
# sns.lineplot(og_train.index[300:1300], og_train['TVOC'][300:1300], alpha=0.7, label='Truth')
# sns.lineplot(train_pred.index[300:1300], train_pred['TVOC'][300:1300], alpha=0.8, label='Pred')
# plt.title('Train Set Truth and Pred Error Comparation',size=20)
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# Test Set evaluation
lstm_ae_model.evaluate(test,test)

In [ ]:
# Original Test DataSet
og_test = pd.DataFrame(X_test_scaled, columns = X_test.columns)
og_test.index = X_test.index
og_test.head()

In [ ]:
# Prediction of Test Set
X_pred_test = lstm_ae_model.predict(test)

test_pred = X_pred_test.reshape(-1, X_pred_test.shape[2])
test_pred = pd.DataFrame(test_pred, columns = X_test.columns)
test_pred.index = og_test[:-2].index
print('test_pred.shape :',test_pred.shape)
test_pred.head()

In [ ]:
# Test Set reconstruction error
test_score = RE_SCORE(og_test, test_pred)
test_score = test_score.set_index(test_score.index)
test_score.head()

In [ ]:
# Train Data Reconstruction Error Threshold
upper, lower= np.quantile(test_score['RE_SCORE'].values,0.75), np.quantile(test_score['RE_SCORE'].values,0.25)
iqr = upper-lower
test_boundary = upper + iqr*1.5
print('Test Data IQR Based boundary :',np.round(test_boundary,4))

In [ ]:
test_score['Threshold'] = test_boundary
test_score.head()

In [ ]:
# Reconstruction Error of Test Set
# fig = px.scatter(test_score, x=test_score.index, y='RE_SCORE', title='Reconstruction Error of Test Set')
# fig.add_trace(go.Scatter(x=test_score.index, y=test_score['Threshold'], name='Threshold',
#                          line=dict(width=5,dash='dash',color=('rgb(237,37,75)'))))

In [ ]:
# Reconstruction Error Distribution of Test Set
fig = px.histogram(test_score, x="RE_SCORE", title ='Reconstruction Error Distribution of Test Set', marginal='box')
fig.add_vline(x=test_boundary, line_width=3, line_dash="dash", line_color="red",annotation_text="Threshold "+str(np.round(test_boundary,4)), annotation_position="bottom right")
fig.show()

In [ ]:
# Test Set Truth and Pred Error Comparation
# plt.figure(figsize=(16,6))
# sns.lineplot(og_test.index[300:1300], og_test['TVOC'][300:1300], alpha=0.7, label='Truth')
# sns.lineplot(test_pred.index[300:1300], test_pred['TVOC'][300:1300], alpha=0.8, label='Pred')
# plt.title('Test Set Truth and Pred Error Comparation',size=20)
# plt.legend(loc='upper right')
# plt.show()

In [ ]:
# Total_score DataFrame Definition
total_score = train_score.copy()
total_score = pd.concat([total_score, test_score])
total_score.drop('Threshold',axis=1,inplace=True)

In [ ]:
# Train Data Reconstruction Error Threshold
upper, lower= np.quantile(total_score['RE_SCORE'].values,0.75), np.quantile(total_score['RE_SCORE'].values,0.25)
iqr = upper-lower
total_boundary = upper + iqr*1.5
print('Total Data IQR Based boundary :',np.round(total_boundary,4))

In [ ]:
# Anomaly Detection
total_score['Threshold'] = total_boundary
total_score['Anomaly'] = total_score['RE_SCORE'] > total_score['Threshold']
print('total_score.shape :',total_score.shape)
total_score.head()

In [ ]:
total_score['Anomaly'].value_counts()

In [ ]:
# fig = px.scatter(total_score, x=total_score.index, y='RE_SCORE', title='Total Reconstruction Error & Threshold')
# fig.add_trace(go.Scatter(x=total_score.index, y=total_score['Threshold'], name='Threshold',
#                          line=dict(width=5,dash='dash',color=('rgb(237,37,75)'))))
# fig.show()

In [ ]:
fig = px.histogram(total_score, x="RE_SCORE", title ='Reconstruction Error Distribution of Total DataSet', marginal='box')
fig.add_vline(x=total_boundary, line_width=3, line_dash="dash", line_color="red",annotation_text="Threshold "+str(np.round(total_boundary,4)), annotation_position="bottom right")
fig.show()

### Deep Compact Clustering

<b> Normal Data Extraction

In [ ]:
train_set = pd.concat([pd.DataFrame(X_train_scaled,columns=X_train.columns,index=X_train.index),train_score],axis=1)
print('train_set.shape :',train_set.shape)
train_set.head(2)

In [ ]:
Threshold = train_boundary
normal = train_set[train_set['RE_SCORE'] <= Threshold]
print('normal.shape :',normal.shape)
normal.head(2)

In [ ]:
arr = normal.drop(['RE_SCORE','Threshold'], axis=1).to_numpy()

# how many rows we can actually use?
n_use = min(arr.shape[0], 10035)
# trim that down to a multiple of 3
n_trim = (n_use // 3) * 3

normal_data = arr[:n_trim].reshape(-1, 3, arr.shape[1])
print(normal_data.shape)

<b> Embedded Feature Extraction

In [ ]:
embedded = pd.DataFrame(encoder.predict(normal_data).reshape(-1,4))
embedded.head(10)

In [ ]:
embedded = embedded.iloc[:,[0,2]].rename(columns={0:'comp1',2:'comp2'})
print('embedded.shape :',embedded.shape)
embedded.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Embedded Features Distribution')
# sns.scatterplot(embedded.iloc[:,0], embedded.iloc[:,1], cmap=plt.get_cmap('Paired'))
# plt.show()

<b> 밀도 기반 군집화 모델 : DBSCAN

In [ ]:
dbscan = DBSCAN(eps = 12, min_samples=200, metric='euclidean')
dbscan_labels = dbscan.fit_predict(embedded)

In [ ]:
embedded

In [ ]:
plt.figure(figsize=(10,7))
plt.title('DBSCAN Clustering')
sns.scatterplot(x=embedded['comp1'], y=embedded['comp2'], hue=dbscan_labels)
plt.show()

In [ ]:
embedded['label'] = dbscan_labels
print('embedded.shape :',embedded.shape)
embedded.head()

In [ ]:
cluster = embedded[embedded['label'] == 0 ].drop('label',axis=1)
print('cluster.shape :',cluster.shape)
cluster.head()

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Normal Cluster Points Distribution')
sns.scatterplot(x=cluster['comp1'], y=cluster['comp2'], cmap=plt.get_cmap('Paired'))
plt.show()

### OC-SVM

In [ ]:
oc_svm = OneClassSVM(gamma='scale', kernel='rbf', max_iter=1000, nu=0.05, verbose=True)
oc_svm.fit(cluster[['comp1','comp2']])

In [ ]:
cluster['label'] = oc_svm.predict(cluster[['comp1','comp2']])
cluster['label'] = np.array([0,0,1])[cluster['label'].values]
print(cluster['label'].value_counts())
cluster.head(1)

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Normal Data OC-SVM Prediction')
# sns.scatterplot(cluster.iloc[:,0], cluster.iloc[:,1], cmap=plt.get_cmap('Paired'), hue=cluster.iloc[:,2])
# plt.show()

In [ ]:
test_feature = pd.DataFrame(encoder.predict(test).reshape(-1,4))
test_feature.head(10)

In [ ]:
test_feature = test_feature.iloc[:,[0,2]].rename(columns={0:'comp1',2:'comp2'})
test_feature.index = test_score.index
print('test_feature.shape :',test_feature.shape)
test_feature.head()

In [ ]:
test_feature['DCC_score'] = oc_svm.score_samples(test_feature[['comp1','comp2']])
test_feature['distance'] = oc_svm.decision_function(test_feature[['comp1','comp2']])
test_feature['DCC_label'] = oc_svm.predict(test_feature[['comp1','comp2']])
test_feature['DCC_label'] = np.array([0,0,1])[test_feature['DCC_label'].values]
test_feature.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Test Data OC-SVM Prediction')
# sns.scatterplot(test_feature['comp1'], test_feature['comp2'], cmap=plt.get_cmap('Paired'), hue=test_feature['DCC_label'])
# plt.show()

In [ ]:
plt.figure(figsize=(10,7))
plt.title('Embedded Features OC-SVM Prediction')
sns.scatterplot(x=test_feature['comp1'], y=test_feature['comp2'], hue=test_feature['DCC_label'])
sns.scatterplot(x=cluster['comp1'], y=cluster['comp2'], color='navy', alpha=0.5)
plt.show()

In [ ]:
# # OC-SVM Model Save
# filename = './oc_svm_model_save/OC-SVM_model.sav'
# pickle.dump(oc_svm, open(filename, 'wb'))

In [ ]:
# # OC-SVM Model Load
# oc_svm = pickle.load(open(filename, 'rb'))

### Decision Rule Definition

<b> 1. Reconstruction Error Based Model

In [ ]:
RE_model = lstm_ae_model

In [ ]:
test_score['RE_label'] = (test_score['RE_SCORE'] > train_boundary).astype('int')
re_score = test_feature[['comp1','comp2']]
re_score = pd.concat([re_score,test_score[['RE_SCORE','RE_label']]],axis=1)
print(re_score.shape)
re_score.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Test Data RE_Based Prediction')
# sns.scatterplot(re_score['comp1'], re_score['comp2'], cmap=plt.get_cmap('Paired'), hue=re_score['RE_label'])
# plt.show()

<b> 2. DCC OC-SVM Based Model

In [ ]:
DCC_model = oc_svm

In [ ]:
dcc_score = test_feature
dcc_score.index = test_score.index
print(dcc_score.shape)
dcc_score.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Test Data OC-SVM Prediction')
# sns.scatterplot(dcc_score['comp1'], dcc_score['comp2'], cmap=plt.get_cmap('Paired'), hue=dcc_score['DCC_label'])
# plt.show()

<b> 3. DCC LSTM-AE Model

In [ ]:
vote_score = pd.concat([re_score, dcc_score[['DCC_score','distance','DCC_label']]],axis=1)
print('vote_score.shape :',vote_score.shape)
vote_score.head()

In [ ]:
print(vote_score['RE_label'].value_counts())
print(vote_score['DCC_label'].value_counts())
print(((vote_score['RE_label'] > 0 )  | (vote_score['DCC_label'] > 0)).value_counts())
print(((vote_score['RE_label'] > 0 )  & (vote_score['DCC_label'] > 0)).value_counts())

In [ ]:
vote_score['Hard Vote OR'] = ((vote_score['RE_label'] > 0 )  | (vote_score['DCC_label'] > 0)).astype(int)
vote_score['Hard Vote AND'] = ((vote_score['RE_label'] > 0 )  & (vote_score['DCC_label'] > 0)).astype(int)
vote_score.head(1)

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Test Data Hard Voter OR')
# sns.scatterplot(vote_score['comp1'], vote_score['comp2'], cmap=plt.get_cmap('Paired'), hue=vote_score['Hard Vote OR'])
# plt.show()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Test Data Hard Voter AND')
# sns.scatterplot(vote_score['comp1'], vote_score['comp2'], cmap=plt.get_cmap('Paired'), hue=vote_score['Hard Vote AND'])
# plt.show()

## 4. Simulation

### Simultation Data Preprocessing

In [ ]:
print(df11.shape)
df11.head(2)

In [ ]:
df11.hist(figsize=(12,12))
plt.show()

In [ ]:
simul_df = pd.read_csv('./data/simul_preprocessed.csv')
simul_df = simul_df.set_index('TsYMD')
simul_df.index.name = 'Time'
print('simul_df.shape :',simul_df.shape)

In [ ]:
print('Missing Value :',simul_df.isna().sum().sum())
simul_df.head(1)

In [ ]:
simul_df.hist(figsize=(12,12))
plt.show()

In [ ]:
plt.hist(simul_df['TVOC'],bins=100)
plt.xlim([0,2000])

In [ ]:
simul_df['CH2O'].sort_values().value_counts()

In [ ]:
simul_len = simul_df.shape[0]

In [ ]:
simul_df.head(1)

In [ ]:
# plt.figure(figsize=(10,6))
# sns.lineplot(np.arange(simul_len),simul_df['CH2O'].values, label='CH2O')
# plt.show()

In [ ]:
# plt.figure(figsize=(10,6))
# sns.lineplot(np.arange(simul_len),simul_df['Temperature'].values, label='Temperature')
# sns.lineplot(np.arange(simul_len),simul_df['Humidity'].values, label='Humidity')
# sns.lineplot(np.arange(simul_len),simul_df['CO'].values, label='CO')
# plt.show()

In [ ]:
# plt.figure(figsize=(18,10))
# sns.lineplot(np.arange(simul_len),simul_df['TVOC'].values, label='TVOC')
# sns.lineplot(np.arange(simul_len),simul_df['CO2'].values, label='CO2')
# sns.lineplot(np.arange(simul_len),simul_df['CH2O'].values, label='CH2O')
# sns.lineplot(np.arange(simul_len),simul_df['PM10'].values, label='PM10')
# plt.show()

In [ ]:
simul_df.info()

In [ ]:
boxplotEDA(simul_df,simul_df.columns,3,3, 'Simulation Data Boxplot')

In [ ]:
simul_norm = MinMaxScaler()
X_simul = simul_norm.fit_transform(simul_df)
print('X_simul.shape :',X_simul.shape)

In [ ]:
simul_size = int(X_simul.shape[0]/3)*3
simul_size

In [ ]:
X_simul = np.reshape(X_simul[:simul_size], (-1,3,7))
print('X_simul.shape :',X_simul.shape)

### Simulation Data Reconstruction Error

In [ ]:
og_train.head()

In [ ]:
# Original Simulation DataSet
og_simul = pd.DataFrame(X_simul.reshape(-1,7), columns = simul_df.columns)
og_simul.index = simul_df[:simul_size].index
og_simul.head()

In [ ]:
# Prediction of Simul DataSet
X_pred_simul = lstm_ae_model.predict(X_simul)

simul_pred = X_pred_simul.reshape(-1, X_pred_simul.shape[2])
simul_pred = pd.DataFrame(simul_pred, columns = simul_df.columns)
simul_pred.index = simul_df[:simul_size].index
print('simul_pred.shape :',simul_pred.shape)
simul_pred.head()

In [ ]:
# Simul data Reconstruction Error
simul_score = RE_SCORE(og_simul, simul_pred)
simul_score = simul_score.set_index(simul_score.index)
simul_score.head()

In [ ]:
# Simul Data Reconstruction Error Threshold
upper, lower= np.quantile(simul_score['RE_SCORE'].values,0.75), np.quantile(simul_score['RE_SCORE'].values,0.25)
iqr = upper-lower
simul_boundary = upper + iqr*1.5
print('Simul Data IQR Based boundary :',np.round(simul_boundary,4))

In [ ]:
simul_score['Threshold'] = train_boundary
simul_score.head()

In [ ]:
simul_score.info()

In [ ]:
# Reconstruction Error of Simul Data
fig = px.scatter(simul_score, x=simul_score.index, y='RE_SCORE', title='Reconstruction Error of Simul Set')
fig.add_trace(go.Scatter(x=simul_score.index, y=simul_score['Threshold'], name='Threshold',
                         line=dict(width=5,dash='dash',color=('rgb(237,37,75)'))))

In [ ]:
# Reconstruction Error Distribution of Simul Data
fig = px.histogram(simul_score, x="RE_SCORE", title ='Reconstruction Error Distribution of Simul Data', marginal='box')
fig.add_vline(x=simul_boundary, line_width=3, line_dash="dash", line_color="red",annotation_text="Threshold "+str(np.round(simul_boundary,4)), annotation_position="bottom right")
fig.show()

In [ ]:
# # Simul Data Truth and Pred Error Comparation
# plt.figure(figsize=(16,6))
# sns.lineplot(np.arange(simul_size), og_simul['TVOC'], alpha=0.7, label='Truth')
# sns.lineplot(np.arange(simul_size), simul_pred['TVOC'], alpha=0.8, label='Pred')
# plt.title('Simul Data Truth and Pred Error Comparation',size=20)
# plt.legend(loc='upper right')
# plt.show()

### Simulation Data Clustering

In [ ]:
embedded_simul = pd.DataFrame(encoder.predict(X_simul).reshape(-1,4))
embedded_simul.head(1)

In [ ]:
embedded_simul = embedded_simul.iloc[:,[0,2]].rename(columns={0:'comp1',2:'comp2'})
print('embedded_simul.shape :',embedded_simul.shape)
embedded_simul.head()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Embedded_Simul Features Distribution')
# sns.scatterplot(embedded_simul.iloc[:,0], embedded_simul.iloc[:,1], cmap=plt.get_cmap('Paired'))
# sns.scatterplot(cluster.iloc[:,0], cluster.iloc[:,1], cmap=plt.get_cmap('Paired'))
# # plt.xlim(0,25)
# # plt.ylim(0,50)
# plt.show()

### Simulation Data Decision Rule Apply

In [ ]:
embedded_simul.index = simul_score.index
simul_score['comp1'] = embedded_simul['comp1']
simul_score['comp2'] = embedded_simul['comp2']
simul_score['RE_label'] = (simul_score['RE_SCORE'] > simul_score['Threshold']).astype('int')
simul_score.head()

In [ ]:
simul_score['RE_label'].value_counts()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Simul Data RE_Based Prediction')
# sns.scatterplot(simul_score['comp1'], simul_score['comp2'], cmap=plt.get_cmap('Paired'), hue=simul_score['RE_label'])
# plt.show()

In [ ]:
embedded_simul['DCC_score'] = oc_svm.score_samples(embedded_simul[['comp1','comp2']])
embedded_simul['distance'] = oc_svm.decision_function(embedded_simul[['comp1','comp2']])
embedded_simul['DCC_label'] = oc_svm.predict(embedded_simul[['comp1','comp2']])
embedded_simul['DCC_label'] = np.array([0,0,1])[embedded_simul['DCC_label'].values]
embedded_simul.head()

In [ ]:
embedded_simul['DCC_label'].value_counts()

In [ ]:
# plt.figure(figsize=(10,7))
# plt.title('Simul Data OC-SVM Prediction')
# sns.scatterplot(embedded_simul['comp1'], embedded_simul['comp2'], cmap=plt.get_cmap('Paired'), hue=embedded_simul['DCC_label'])
# plt.show()

In [ ]:
vote_score.head()